Importing necessary libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import re
import json

Setting up the base URL

In [ ]:
base_url = "https://timesofindia.indiatimes.com/2020/1/1/archivelist/year-2020,month-1,starttime-43831.cms"

Fetching the content

In [ ]:
response = requests.get(base_url)
html_content = response.content

#BeautifulSoup to parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')



Extracting the following necessary info :
Title
Date
Full text
Summary (if available)
Tags (if available)
Keywords (if available)
url

In [ ]:
span_tags = soup.find_all('span')
date_pattern = re.compile(r"(\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4},\s+\d{2}:\d{2}\s+\w{3}\b)")
articles_data = []
if response.status_code == 200:

#parsing the html content

    soup = BeautifulSoup(response.content, 'html.parser')

#finding the links
    article_links = soup.find_all("a", href=True)

#Iterating through article links and extracting information
    for link in article_links:
        article_url = link['href']

#check to ensure its an article link
        if article_url.startswith("http://timesofindia.indiatimes.com/"):
 #print("Fetching content for:", article_url)

# Fetching the article content
            article = Article(article_url)
            article.download()
            article.parse()
            article.nlp()

# Extracting necessary info information
            title = article.title
            date = article.publish_date
            full_text = article.text
            summary = article.summary if article.summary else "Summary not available"
            keywords = article.keywords
            tags = article.tags if article.tags else []
# Displaying the extracted information
            print("Title:", title)
            for tag in span_tags:
                if tag.text.strip():
                   match = date_pattern.search(tag.text.strip())
                   if match:
                       print("Date:", match.group())
            print("Summary:", summary)
            print("Tags:", tags)
            print("Keywords:", keywords)
            print("URL:", article_url)
            print("--------------------------------------------------")

# Break after printing the first article
            break

else:
    print("Failed to fetch the archive page.")


In [ ]:
articles_data = []

if response.status_code == 200:
#links to archive pages
    article_links = soup.find_all("a", href=True)

#Iterating through article links and extracting info
    for link in article_links:
        article_url = link['href']

# Checking if it is an article link
        if article_url.startswith("http://timesofindia.indiatimes.com/"):
            print("Fetching content for:", article_url)

# Fetching the article content
            article = Article(article_url)
            article.download()
            article.parse()
            article.nlp()

# Extracting additional information
            title = article.title
            full_text = article.text
            summary = article.summary if article.summary else "Summary not available"
            keywords = list(article.keywords)  # Convert set to list
            tags = list(article.tags)  # Convert set to list

# Extracting the date from span tags
            date = None
            for tag in span_tags:
                if tag.text.strip():
                    match = date_pattern.search(tag.text.strip())
                    if match:
                        date = match.group()
                        break

# Store article information in a dictionary
            article_info = {
                "Title": title,
                "Date": date,
                "Summary": summary,
                "Tags": tags,
                "Keywords": keywords,
                "URL": article_url
            }

# Appending article information to the list
            articles_data.append(article_info)

# Break after fetching the first article
            break

else:
    print("Failed to fetch the archive page.")



Saving the data in a JSON file

In [ ]:
# Path to save the JSON file
json_file_path = "C:\\Users\\Anusha Gadgil\\Desktop\\RA stuff\\January_2020.json"

# Write the article data to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(articles_data, json_file, indent=4)

print("Data saved to:", json_file_path)